In [1]:
%pip install git+https://github.com/kmayerb/tcrdist3.git@0.2.2

  Cloning https://github.com/kmayerb/tcrdist3.git (to revision 0.2.2) to /private/var/folders/vx/p6h8zj0n0wxd9qrn2pk09l9c0000gn/T/pip-req-build-iq79_jvo
  Running command git clone -q https://github.com/kmayerb/tcrdist3.git /private/var/folders/vx/p6h8zj0n0wxd9qrn2pk09l9c0000gn/T/pip-req-build-iq79_jvo
  Running command git checkout -q 04b0b8c2573d04a9d2cb77f7a3aeeed3a0eab167
  Resolved https://github.com/kmayerb/tcrdist3.git to commit 04b0b8c2573d04a9d2cb77f7a3aeeed3a0eab167
     |████████████████████████████████| 3.2 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 366 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 92 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 4.3 MB/s eta 0:00:011
     |████████████████████████████████| 27.1 MB 14.9 MB/s eta 0:00:01
  Created wheel for tcrdist3: filename=tcrdist3-0.2.2-py3-none-any.whl size=2101181 sha256=825f6f288e308dca4

In [2]:
from tcrdist.repertoire import TCRrep
import pwseqdist as pw
from tcrdist.rep_funcs import _pws, _pw

from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pickle

In [4]:
import pandas as pd
df = pd.read_csv('/Users/joesouber/OneDrive - University of Bristol/MSc Data Science/data science mini project/dsmp-2024-group-13/vdjdb_full.txt', delimiter = '\t')



/var/folders/vx/p6h8zj0n0wxd9qrn2pk09l9c0000gn/T/ipykernel_8216/3406053389.py:2: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/joesouber/OneDrive - University of Bristol/MSc Data Science/data science mini project/dsmp-2024-group-13/vdjdb_full.txt', delimiter = '\t')


In [5]:
# Assuming 'df' is your original DataFrame.

# Remove duplicate rows based on all columns and overwrite the original DataFrame with the deduplicated one.
vdjdb = df.drop_duplicates()

# Display the shape of the DataFrame (number of rows, number of columns) after deduplication.
vdjdb.shape

# Find and print the unique values in the 'antigen.gene' column.
# This helps identify all the unique genes present in the dataset after removing duplicates.
unique_genes = vdjdb['antigen.gene'].unique()
print(unique_genes)

# Store the number of rows (TCRs) in the DataFrame after deduplication.
n_orig = vdjdb.shape[0]

# Print a summary message that shows:
# 1. The total number of TCRs in the deduplicated dataset.
# 2. The number of unique epitopes identified by looking at unique values in the 'antigen.gene' column.
print(f'The dataset contains {vdjdb.shape[0]} TCRs for {len(vdjdb["antigen.gene"].unique())} different epitopes. The amount of TCRs per epitope:\n')

# Display the count of TCRs for each unique epitope in the 'antigen.gene' column.
# This uses the value_counts() method, which counts the occurrence of each unique value in the column,
# effectively showing the distribution of TCRs across different epitopes.
vdjdb['antigen.gene'].value_counts()


['Nef' 'Gliadin' 'pp65' 'Nucleocapsid' 'ORF3' 'ORF1ab' 'Spike' 'NDC1'
 'TKT' 'SEC24A' 'AKAP13' 'EXOC8' 'PABPC1' 'MLANA' 'BRLF1' 'Gag' 'CFP10'
 'Rv1195' 'Rv3804c' 'Rv0288' 'Rv3874' 'PPE33' 'Rv3616c' 'Rv1388' 'IE1'
 'EBNA1' 'BZLF1' 'Tax' 'Ovotransferrin' 'Kctd20' 'HA' 'Synthetic' 'Gtpbp1'
 'EBNA3A' 'Ndufa4' 'N' 'M' 'Mbp' 'MBP' 'NY-ESO-1' 'UL40' 'TPI1' 'Rbm5'
 'EBNA6' 'Tel1' 'ABCD3' 'LYZ' 'BMLF1' 'PA' 'ELAVL4' 'MCC' 'INS' nan 'Pol'
 'TERT' 'NS3' 'EMC' 'synthetic' 'Der' '3K' 'QL9' 'MCC-P5E' 'Mimotope'
 'TRM3' 'PCPB9' 'PCPB7' 'PCPC5' 'PCPE3' 'PCPA12' 'MIM2' 'WT1' 'MAGE-A3'
 'TITIN' 'M1-F5L' 'M1-G4E' 'P18-I10' 'DQ8-GLIA-ALPHA1' 'DQ8.5-GLIA-GAMMA1'
 'Gp33(Y4A)' 'Gp33(PA)' 'Gp33(PF)' 'GAG' 'NP' 'Gp33' 'Gp33-41'
 'Leader peptide' 'GAP50' 'RT' 'Velcro' 'P24' '8F10' 'DBH' 'PADI4' 'NP338'
 'GLUT-L1' 'ANKRD30A' 'MAGE-A4' 'MEROPS' 'OPT' 'PMEL' 'P53' 'ENR' 'TP53'
 'PIK3CA' 'DQ2-GLIA-OMEGA1' 'NS4B' 'BST2' 'IE2' 'M2-1' 'UL49' 'NSP3'
 'INS-DRiP' 'PTPRN' 'EBNA4' 'EBNA3B' 'pp50' 'E6' 'KRAS' 'FGFR3' 'LMP2A

IE1         14126
pp65         8595
M            7891
Gag          3372
Spike        2964
            ...  
CRISPLD1        1
FANCI           1
WDR87           1
PDE4A           1
NY-ESO-5        1
Name: antigen.gene, Length: 268, dtype: int64

In [6]:
#cleaning, renaming and organising df

df_slim = vdjdb.iloc[:,[0,1,2,3,4,6,7,12]]
df_cleaned = df_slim.dropna(subset = ['cdr3.alpha','cdr3.beta'])
df_cleaned.rename(columns={'cdr3.alpha': 'cdr3_a_aa', 'cdr3.beta': 'cdr3_b_aa', 'v.alpha':'v_a_gene','j.alpha': 'j_a_gene','v.beta': 'v_b_gene','j.beta': 'j_b_gene','antigen.gene':'epitope'}, inplace=True)

df_cleaned

/var/folders/vx/p6h8zj0n0wxd9qrn2pk09l9c0000gn/T/ipykernel_8216/3294306157.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={'cdr3.alpha': 'cdr3_a_aa', 'cdr3.beta': 'cdr3_b_aa', 'v.alpha':'v_a_gene','j.alpha': 'j_a_gene','v.beta': 'v_b_gene','j.beta': 'j_b_gene','antigen.gene':'epitope'}, inplace=True)


,cdr3_a_aa,v_a_gene,j_a_gene,cdr3_b_aa,v_b_gene,j_b_gene,species,epitope
0,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,Nef
2,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,Nef
3,CAVKASGSRLT,TRAV2*01,NaN,CASSYEPGQVSHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,Nef
4,CAYRPPGTYKYIF,TRAV38-2/DV8*01,TRAJ40*01,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,HomoSapiens,Nef
5,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,Nef
...,...,...,...,...,...,...,...,...
62172,CMDEGGSNYKLTF,TRAV26-1*01,TRAJ53*01,CASSVRSTDTQYF,TRBV7-2*01,TRBJ2-3*01,HomoSapiens,Gluten
62173,CSLYNNNDMRF,TRAV26-1*01,TRAJ43*01,CASSLRYTDTQYF,TRBV7-2*01,TRBJ2-3*01,HomoSapiens,Gluten
62174,CALSTDSWGKLQF,TRAV6*01,TRAJ24*01,CASSPGQGGDNEQFF,TRBV7-3*01,TRBJ2-1*01,HomoSapiens,Gluten
62175,CAPQGATNKLIF,TRAV12-2*01,TRAJ32*01,CASSLGAGGQETQYF,TRBV5-1*01,TRBJ2-5*01,HomoSapiens,Gluten


In [7]:
# <chains> - tcrdist3 accepts ['alpha', 'beta'] or ['alpha'] or ['beta']
chains = ['alpha', 'beta']
# <cdr3> We can choose either 'cdr3' or 'all'. Choosing 'all' will
# compute distances using CDR1,CDR2, and CDR2.5 as well as CDR3
cdrs = 'all'

In [8]:
# Initialize a list with the column names to consider when identifying duplicates to drop.
# Initially, it includes only the 'epitope' column.
drop_dup_cols = ['epitope']

# Add columns related to the alpha chain of the TCR if certain conditions are met.
# This depends on two factors: whether the 'alpha' chain is part of the analysis (indicated by 'alpha' in chains)
# and the level of detail being considered (CDR3 only or all features).
if 'alpha' in chains and cdrs=='cdr3':
    drop_dup_cols += ['cdr3_a_aa']  # Add only the CDR3 alpha amino acid sequence for duplication criteria.
elif 'alpha' in chains and cdrs=='all':
    drop_dup_cols += ['cdr3_a_aa', 'v_a_gene', 'j_a_gene']  # Add alpha chain CDR3, V gene, and J gene.

# Similarly, add columns related to the beta chain of the TCR if certain conditions are met.
if 'beta' in chains and cdrs=='cdr3':
    drop_dup_cols += ['cdr3_b_aa']  # Add only the CDR3 beta amino acid sequence for duplication criteria.
elif 'beta' in chains and cdrs=='all':
    drop_dup_cols += ['cdr3_b_aa', 'v_b_gene', 'j_b_gene']  # Add beta chain CDR3, V gene, and J gene.

# Drop duplicate rows based on the specified columns in 'drop_dup_cols'.
# 'keep='first'' means only the first occurrence is kept; subsequent duplicates are dropped.
# The result is copied to ensure a new DataFrame object is created.
df = df_cleaned.drop_duplicates(subset=drop_dup_cols, keep='first').copy()

# Reset the index of the DataFrame to reflect the dropped rows, dropping the old index.
df.reset_index(drop=True, inplace=True)

# Print the number of duplicates dropped based on the specified criteria.
# This is calculated by comparing the original number of rows before dropping duplicates
# to the number of rows after the operation.
print(f'Dropped {n_orig - df.shape[0]} duplicate TCRs by columns {drop_dup_cols}.')

# Print a summary showing the new distribution of TCRs per epitope after duplicates have been dropped.
print(f'The new amount of TCRs per epitope:\n')
df['epitope'].value_counts()



Dropped 32187 duplicate TCRs by columns ['epitope', 'cdr3_a_aa', 'v_a_gene', 'j_a_gene', 'cdr3_b_aa', 'v_b_gene', 'j_b_gene'].
The new amount of TCRs per epitope:



IE1         13984
EBNA4        2330
M            1949
Spike        1478
BZLF1        1242
            ...  
RFC5            1
PDS5A           1
PCPB7           1
PCPC5           1
NY-ESO-6        1
Name: epitope, Length: 227, dtype: int64

In [9]:
# Initialize a TCRrep instance with the given DataFrame 'df'.
# The 'chains' parameter specifies the TCR chains (alpha and/or beta) to consider,
# and 'deduplicate=True' indicates that duplicate TCRs should be removed in the process.
tr = TCRrep(cell_df=df,
            chains=chains,
            deduplicate=True)

# Initialize 'X' to None. 'X' will later store the matrix of distances,
# which is calculated based on the selected 'chains' and 'cdrs' criteria.
X = None

# If the analysis is to include all features of the TCR ('cdrs' set to 'all'),
# then the pairwise distances are calculated differently depending on the chains involved.
if cdrs == 'all':
    # For both alpha and beta chains, combine the pairwise distance matrices for both chains.
    if chains == ['alpha', 'beta'] or chains == ['beta', 'alpha']:
        X = pd.DataFrame(tr.pw_alpha + tr.pw_beta)
    # For alpha chain only, use the alpha chain pairwise distance matrix.
    elif chains == ['alpha']:
        X = pd.DataFrame(tr.pw_alpha)
    # For beta chain only, use the beta chain pairwise distance matrix.
    elif chains == ['beta']:
        X = pd.DataFrame(tr.pw_beta)

# If the analysis is focused on the CDR3 region only ('cdrs' set to 'cdr3'),
# then the pairwise distances are calculated based on CDR3 sequences.
if cdrs == 'cdr3':
    # For both alpha and beta chains, combine the pairwise distance matrices for CDR3 regions of both chains.
    if chains == ['alpha', 'beta'] or chains == ['beta', 'alpha']:
        X = pd.DataFrame(tr.pw_cdr3_a_aa + tr.pw_cdr3_b_aa)
    # For alpha chain only, use the CDR3 alpha region pairwise distance matrix.
    elif chains == ['alpha']:
        X = pd.DataFrame(tr.pw_cdr3_a_aa)
    # For beta chain only, use the CDR3 beta region pairwise distance matrix.
    elif chains == ['beta']:
        X = pd.DataFrame(tr.pw_cdr3_b_aa)

# Assert that 'X' is not None to ensure that a distance matrix has been assigned.
# This acts as a check to ensure that the preceding logic correctly assigned a distance matrix
# based on the specified 'chains' and 'cdrs' parameters.
assert X is not None


/Users/joesouber/opt/anaconda3/lib/python3.9/site-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/Users/joesouber/opt/anaconda3/lib/python3.9/site-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")
/Users/joesouber/opt/anaconda3/lib/python3.9/site-packages/tcrdist/repertoire.py:500: UserWarning: TRAV26-1*01 gene was not recognized in reference db no cdr seq could be inferred
  f0 = lambda v : self._map_gene_to_reference_seq2(gene = v,
/Users/joesouber/opt/anaconda3/lib/python3.9/site-packages/tcrdist/repertoire.py:500: UserWarning: TRAV38-2/DV8*01 gene was not recognized in reference db no cdr seq could be inferred
  f0 = lambda v : self._map_gene_to_reference_seq2(gene = v,
/Users/joesouber/opt/anaconda3/lib/python3.9/site-packages/tcrdist/repertoire.py:500: UserWarning: TRAV

In [10]:
X

,0,1,2,3,4,5,6,7,8,9,...,4426,4427,4428,4429,4430,4431,4432,4433,4434,4435
0,0,280,212,279,271,265,265,284,350,325,...,274,171,237,237,237,239,223,450,266,251
1,280,0,250,244,57,239,267,276,302,261,...,235,293,302,302,302,305,266,439,233,238
2,212,250,0,189,235,236,217,220,287,265,...,276,197,215,215,215,216,186,451,212,162
3,279,244,189,0,241,249,207,229,286,280,...,295,233,302,302,302,293,264,418,177,272
4,271,57,235,241,0,215,258,264,290,258,...,244,287,287,287,287,284,254,424,221,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4431,239,305,216,293,284,321,305,308,341,340,...,260,212,233,233,233,0,243,360,278,274
4432,223,266,186,264,254,252,272,254,291,296,...,266,208,232,232,232,243,0,465,231,172
4433,450,439,451,418,424,470,469,454,449,434,...,400,422,473,473,473,360,465,0,436,415
4434,266,233,212,177,221,251,237,238,318,274,...,292,236,287,287,287,278,231,436,0,212
